Приведите по 2 примера, когда лучше максимизировать Precision, а когда Recall.

1) Организация - сеть розничных магазинов. Она проводит эксперимент с новым товаром. В итоге мы классифицируем магазины следующим образом: 1 - магазин продаст новинку, 0 - магазин не продаст новинку.
Если у компании текущая стратегия заточена на быстрое развитие, не смотря, на возможно замороженные деньги, или товар высокомаржинальный и не дорогой, то лучше максимизировать Recall, дабы охватить максимальное количество клиентов новинкой.
Если у компании текущая стратегия заточена на оптимизацию с минимальными вложениями в новые проекты, или же товар дорогой и низкомаржинальный, то лучше максимизировать Precision, дабы был меньше шанс заморозить деньги в товаре.

2) Большой адронный коллайдер - проводятся эксперементы по столкновению различных частиц. В результате столкновения проводится измерение параметров у частиц, которые вылетают из столкновений. Наша модель на основании параметров вылетевшей частицы говорит: 1 - неивестная(новая) частица, 0 - известная частица. Лучше максимизировать Recall дабы не пропустить возможных новых открытий :)

3) Камера фиксации правонарушений - на основании видеопотока определяет совершает ли правонарушение водитель. Классификация 1 - правонарушение зафиксировано, 0 - правонарушение не зафиксировано.
Если городу нужно больше денег, и мы не заморачиваемся по поводу того, что людям нужно пройти 7 кругов ада,чтобы доказать отсутствие правонарушения - мы максимизируем Recall. Если же мы все таки заботимся о гражданах и не хотим, чтоб им прилетали случайные штрафы - максимизируем Precision.

Почему мы используем F-меру, почему, например, нельзя просто взять среднее от Precision и Recall?

Когда один из показателей будет низким, например один = 0.1, второй = 0.9,
то в случае ср. арефметического мы получим 0.45, а F-мера будет равна 0.18.
Т.о. мы страхуемся от сильно низких значениях в одном параметре и в другом, что может показывать, что модель переобучена.
Плюс можно варьировать показатель beta, что позволит сосредоточиться на одной из двух мер в большей степени.

In [1]:
import numpy as np #взял тестовый пример, чтобы на живых данных посчитать
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

In [2]:
df = pd.read_csv('training_project_data.csv')

In [3]:
for colname in ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']:
    df[colname] = df[colname].astype(str)

In [4]:
df.loc[df['EDUCATION'] == '0', 'EDUCATION'] = df['EDUCATION'].mode()[0]
df.loc[df['MARRIAGE'] == '0', 'MARRIAGE'] = df['MARRIAGE'].mode()[0]

In [5]:
df['ID'] = df.index.tolist()

In [6]:
df['IS_MALE'] = df['SEX'].map({'1':'1', '2':'0'}).astype(int)
for cat_colname in df.select_dtypes(include='object').columns[1:]:
    df = pd.concat([df, pd.get_dummies(df[cat_colname], prefix=cat_colname)], axis=1)

In [7]:
X = df.drop(columns='NEXT_MONTH_DEFAULT')
y = df['NEXT_MONTH_DEFAULT']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=2)
X_train.shape, X_test.shape

((8000, 61), (2000, 61))

In [8]:
%%time
tree = DecisionTreeClassifier(random_state=1)

param_grid=[{
             'max_depth': range(1, 7), 
             'min_samples_leaf': range(1, 7), 
            }]

grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
tree = grid_search.best_estimator_

Wall time: 9.39 s


In [9]:
pred_train = tree.predict(X_train)
pred_test = tree.predict(X_test)

pred_proba_test = tree.predict_proba(X_test)
pred_proba_test[:5]

array([[0.89156047, 0.10843953],
       [0.89156047, 0.10843953],
       [0.89156047, 0.10843953],
       [0.89156047, 0.10843953],
       [0.79866332, 0.20133668]])

Реализовать функции для подсчета Accuracy, Precision, Recall, F-score, которые на вход принимают y_true (истинные значения), y_pred (предсказанные значения), а на выход дается метрика.

In [10]:
def pred(x, y):
    def pred2(z): #Скорее всего можно проще, но с df работать удобнее)
        z = pd.DataFrame(z).reset_index()
        z.drop('index', axis= 1, inplace=True)
        z.columns=['value']
        return z
    x = pred2(x)
    y = pred2(y)
    if len(x) != len(y):
        raise Exception('x and y has different len')
    else:
        return x.merge(y, left_index=True, right_index=True)

In [11]:
def accuracy(y_true, y_pred):
    y = pred(y_true, y_pred)
    return len(y[y['value_x'] == y['value_y']]) / len(y)

In [12]:
def precision(y_true, y_pred, value=1): # value - значение для которого будем считать соответствующую метрику
    y = pred(y_true, y_pred)
    tp = len(y[(y['value_x'] == value) & (y['value_y'] == value)])
    fp = len(y[(y['value_x'] != value) & (y['value_y'] == value)])
    return  tp / (tp + fp)
    

In [13]:
def recall(y_true, y_pred, value=1): # value - значение для которого будем считать соответствующую метрику
    y = pred(y_true, y_pred)
    tp = len(y[(y['value_x'] == value) & (y['value_y'] == value)])
    fn = len(y[(y['value_x'] == value) & (y['value_y'] != value)])
    return  tp / (tp + fn)

In [14]:
def f_score(y_true, y_pred, value=1, beta=1): # value - значение для которого будем считать соответствующую метрику
    prec = precision(y_true, y_pred, value)
    rec = recall(y_true, y_pred, value)
    return  (1 + beta ** 2) * prec * rec / (beta ** 2 * prec + rec )

In [15]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.84      0.95      0.89      1587
           1       0.62      0.33      0.43       413

    accuracy                           0.82      2000
   macro avg       0.73      0.64      0.66      2000
weighted avg       0.80      0.82      0.80      2000



In [16]:
print(f'Accuracy: {accuracy(y_test, pred_test):.2f}')
print(f'Precision: {precision(y_test, pred_test):.2f}')
print(f'Recall: {recall(y_test, pred_test):.2f}')
print(f'F-score: {f_score(y_test, pred_test):.2f}')

Accuracy: 0.82
Precision: 0.62
Recall: 0.33
F-score: 0.43


In [17]:
print(f'Accuracy: {accuracy(y_test, pred_test):.2f}')
print(f'Precision: {precision(y_test, pred_test, value=0):.2f}')
print(f'Recall: {recall(y_test, pred_test, value=0):.2f}')
print(f'F-score: {f_score(y_test, pred_test, value=0):.2f}')

Accuracy: 0.82
Precision: 0.84
Recall: 0.95
F-score: 0.89
